In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case2848rte

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_10290/2379256297.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
random.seed(10)
np.random.seed(20)

In [3]:
# Load any case file you need
net = case2848rte()

# Same settings as in the synthetic data generation
net.sgen.in_service = False
net.gen.max_p_mw = 500
net.line.max_loading_percent = 10000
net.trafo.max_loading_percent = 1e20
net.ext_grid.min_p_mw = 0

/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packag

In [4]:
# # Change bus name from 'Name' to 'Bus_no'
# net.bus.name.name = 'Bus_no'
# # Bus no. starts from 0
# net_index = net.bus.name.astype(int) - 1

net_index = np.arange(net.bus.shape[0])

#### Get node features from the case network

In [5]:
# Create DataFrame to store network data
network = pd.DataFrame()

# Set network index
network.index = net_index

In [6]:
# Get features from 'bus
network['vn_kv'] = net.bus.vn_kv.values

# Get features from 'load'
net.load.index = net.load.bus
network['load_p_mw'] = net.load.p_mw

# Get features from 'gen'
net.gen.index = net.gen.bus
network['max_gen_p_mw'] = net.gen.max_p_mw
network['min_gen_p_mw'] = net.gen.min_p_mw

# Get features from 'ext_grid' (ext_grid bus no: 68)
net.ext_grid.index = net.ext_grid.bus
network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
network['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

# Get features from 'poly_cost'
# This needs manual work
gen_price = pd.read_csv('./gen_price.csv', index_col=0, header=0)
network['price_1'] = gen_price.price_term1
network['price_2'] = gen_price.price_term2

# Additional features
network['x_pos'] = net.bus_geodata.x
network['y_pos'] = net.bus_geodata.y

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_10290/3286604619.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
/var/folders/0t/9z26630n32l3qw5w80zvv

FileNotFoundError: [Errno 2] No such file or directory: './gen_price.csv'

#### Get edge index and attributes

In [ ]:
branch = net.line[['from_bus', 'to_bus']]

In [ ]:
branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]

In [ ]:
trafo = net.trafo[['hv_bus', 'lv_bus']]

In [ ]:
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]

#### Save node and edge informartion

In [ ]:
network.to_csv('network/node_features.csv')
branch.to_csv('network/branch_index.csv')
branch_attr.to_csv('network/branch_attr.csv')
trafo.to_csv('network/trafo_index.csv')
trafo_attr.to_csv('network/trafo_attr.csv')

#### Save normalized data and scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Normalize branch atrributes
branch_attr_scaler = MinMaxScaler()
branch_attr_scaled = branch_attr_scaler.fit_transform(branch_attr)
pd.DataFrame(branch_attr_scaled).to_csv('network/branch_attr_scaled.csv')
joblib.dump(branch_attr_scaler, 'network/branch_attr_scaler.scaler')

# Normalize transformer attributes
trafo_attr_scaler = MinMaxScaler()
trafo_attr_scaled = trafo_attr_scaler.fit_transform(trafo_attr)
pd.DataFrame(trafo_attr_scaled).to_csv('network/trafo_attr_scaled.csv')
joblib.dump(trafo_attr_scaler, 'network/trafo_attr_scaler.scaler')

['network/trafo_attr_scaler.scaler']